In [ ]:
# Cài đặt Thư viện Frontend
!pip install streamlit requests pyngrok
print("Installation complete.")

In [ ]:
%%writefile app.py
import streamlit as st
import requests
from datetime import date, timedelta
import json
import os

# === CẬP NHẬT ENDPOINT API ===
# Đổi link trước /generate_itinerary
API_ENDPOINT = "https://four-geckos-juggle.loca.lt/generate_itinerary" 

## --- Login và History ---
if 'logged_in' not in st.session_state:
    st.session_state.logged_in = False
    st.session_state.username = ""
    st.session_state.history = [] 

def login(username):
    if username:
        st.session_state.logged_in = True
        st.session_state.username = username
        st.success(f"Chào mừng, {username}!")
    else:
        st.error("Vui lòng nhập Tên đăng nhập.")

def logout():
    st.session_state.logged_in = False
    st.session_state.username = ""
    st.session_state.history = []
    st.rerun()

## --- Hàm gọi API LLM ---
def generate_itinerary_api_call(origin, destination, start_date, end_date, interests, pace):
    if "http" not in API_ENDPOINT:
        return {"status": "error", "message": "Lỗi cấu hình: API_ENDPOINT chưa hợp lệ."}
        
    payload = {
        "origin": origin,
        "destination": destination,
        "start_date": start_date.isoformat(),
        "end_date": end_date.isoformat(),
        "interests": interests,
        "pace": pace
    }
    
    try:
        # Timeout 300 giây (5 phút)
        response = requests.post(API_ENDPOINT, json=payload, timeout=300) 
        response.raise_for_status() 
        return response.json()
    except requests.exceptions.RequestException as e:
        return {"status": "error", "message": f"API Error: Không thể kết nối đến Backend. Vui lòng kiểm tra API Endpoint và đảm bảo Notebook Backend đang chạy. Chi tiết lỗi: {e}"}

## --- Giao diện Streamlit ---

st.set_page_config(layout="wide", page_title="AI Travel Itinerary Generator")

if not st.session_state.logged_in:
    st.title("Đăng nhập vào AI Travel Planner")
    
    with st.form("login_form"):
        username = st.text_input("Tên đăng nhập")
        submit_button = st.form_submit_button("Đăng nhập", type="primary")
        
        if submit_button:
            login(username)
else:
    with st.sidebar:
        st.header(f"Chào mừng, {st.session_state.username}!")
        st.button("Đăng xuất", on_click=logout)
        
        st.subheader("Chi tiết Chuyến đi")
        
        origin = st.text_input("Origin city", value="Hà Nội")
        destination = st.text_input("Destination city", value="Đà Nẵng")
        
        col1, col2 = st.columns(2)
        with col1:
            start_date = st.date_input("Start", date.today())
        with col2:
            end_date = st.date_input("End", date.today() + timedelta(days=1))
            
        interests = st.multiselect(
            "Interests",
            ['food', 'museums', 'nature', 'nightlife', 'shopping', 'history', 'adventure'],
            default=['food', 'nature']
        )
        
        pace = st.radio(
            "Pace",
            ['Relaxed', 'Normal', 'Tight'],
            index=1
        )
        
        if st.button("Tạo hành trình", type="primary"):
            if not destination:
                st.error("Vui lòng nhập Thành phố Đến.")
            elif start_date > end_date:
                st.error("Ngày bắt đầu phải trước hoặc bằng ngày kết thúc.")
            else:
                with st.spinner("Đang tạo hành trình..."):
                    result = generate_itinerary_api_call(origin, destination, start_date, end_date, interests, pace)
                    
                    st.session_state.history.append({
                        "request": {"dest": destination, "start": start_date, "end": end_date, "pace": pace},
                        "response": result
                    })
                    st.rerun()

    # --- Display Itinerary & History ---
    
    st.title("Hành trình Du lịch của bạn")
    
    # Hiển thị hành trình mới nhất
    if st.session_state.history:
        latest_response = st.session_state.history[-1]['response']
        if latest_response['status'] == 'success':
            latest_itinerary = latest_response['itinerary']
            
            if 'itinerary' in latest_itinerary and latest_itinerary['itinerary']:
                # Lấy ngày đầu và cuối từ JSON
                first_date = latest_itinerary['itinerary'][0].get('date', 'N/A')
                last_date = latest_itinerary['itinerary'][-1].get('date', 'N/A')
                
                st.subheader(f"Hành trình cho {first_date} - {last_date}")
                
                for day_data in latest_itinerary.get('itinerary', []):
                    day_num = day_data.get('day', 'N/A')
                    day_date = day_data.get('date', 'N/A')
                    
                    with st.expander(f"** Ngày {day_num}: {day_date}**", expanded=True):
                        for activity in day_data.get('activities', []):
                            st.markdown(f"** {activity.get('time', 'N/A')}: {activity.get('place', 'N/A')}**")
                            st.markdown(f"*- {activity.get('explanation', 'N/A')}*")
                        st.markdown("---")
            else:
                st.warning("LLM tạo ra JSON thành công nhưng không có dữ liệu hành trình hợp lệ. Hãy thử lại.")
                
        elif latest_response['status'] == 'error':
            st.error("Lỗi khi tạo hành trình.")
            st.error(f"Thông báo lỗi: {latest_response['message']}")
            if 'raw_output' in latest_response['itinerary']:
                 st.code(latest_response['itinerary']['raw_output'], language='text')
            
    st.markdown("---")
    
    # Hiển thị lịch sử
    st.subheader(" Lịch sử tạo Hành trình")
    history_to_display = st.session_state.history[:-1]
    history_to_display.reverse()
    
    if history_to_display:
        for i, item in enumerate(history_to_display): 
            with st.expander(f"Lịch sử {len(history_to_display) - i}: {item['request']['dest']} ({item['request']['start']} - {item['request']['end']})"):
                if item['response']['status'] == 'success':
                    st.json(item['response']['itinerary'], expanded=False)
                else:
                    st.warning(f"Lỗi trong lần tạo này: {item['response']['message']}")
    else:
        st.info("Chưa có lịch sử tạo hành trình nào.")

In [ ]:
# Chạy Streamlit ngầm và thoát
# Dọn cổng
!fuser -k 8501/tcp

import subprocess
import time
import os

STREAMLIT_PORT = 8501

print(f"Starting Streamlit app.py on port {STREAMLIT_PORT} using subprocess...")
streamlit_process = subprocess.Popen(
    ['streamlit', 'run', 'app.py', '--server.port', str(STREAMLIT_PORT)],
    preexec_fn=os.setsid 
)
print(f"Streamlit is running (PID: {streamlit_process.pid}).")
print("Streamlit đã khởi động ngầm. Chuyển sang Cell 4 để tạo LocalTunnel.")

In [ ]:
# Mật khẩu cho LocalTunnel
import requests
tunnel_password = requests.get('https://loca.lt/mytunnelpassword').text.strip()
print(f"Sử dụng IP sau làm mật khẩu: {tunnel_password}")

In [ ]:
# Chạy LocalTunnel và lấy mật khẩu

import requests
import time
time.sleep(2)

print("Running LocalTunnel. Vui lòng giữ Cell này chạy.")
get_ipython().system('npx --yes localtunnel --port 8501')